In [126]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [127]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std) # create a tensor (same shape with input) ,each element is sampled from normal Guaissian.
        # std here as input only influence the shape of eps, rather than define guassian 's sigma!!! 在下面测试了
        return eps.mul(std).add_(mu) # return z sample   # 从 标准高斯 映射到 普通高斯。
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [128]:
# std_dev = 10000000000.0
# input_tensor = torch.tensor(std_dev)

# # 使用 torch.randn_like 创建一个具有相同形状的随机张量，但标准差为 input_tensor 的值
# random_tensor = torch.randn_like(input_tensor)

# print(random_tensor)

In [129]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [130]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')       # 二值分布下 重构损失算出来正比于二进制交叉熵
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())               # 两高斯分布间的KL 散度，有公式。 .exp() 即以e为base，log_var 为指数，结果就是 var！
    return BCE + KLD

In [131]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()     
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [132]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [133]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.045859
Train Epoch: 1 [10000/60000 (17%)]	Loss: 190.692734
Train Epoch: 1 [20000/60000 (33%)]	Loss: 171.830957
Train Epoch: 1 [30000/60000 (50%)]	Loss: 173.555645
Train Epoch: 1 [40000/60000 (67%)]	Loss: 176.866855
Train Epoch: 1 [50000/60000 (83%)]	Loss: 158.913096
====> Epoch: 1 Average loss: 180.6820
====> Test set loss: 163.8706
Train Epoch: 2 [0/60000 (0%)]	Loss: 163.061055
Train Epoch: 2 [10000/60000 (17%)]	Loss: 168.777891
Train Epoch: 2 [20000/60000 (33%)]	Loss: 164.829199
Train Epoch: 2 [30000/60000 (50%)]	Loss: 160.373047
Train Epoch: 2 [40000/60000 (67%)]	Loss: 154.356309
Train Epoch: 2 [50000/60000 (83%)]	Loss: 161.251777
====> Epoch: 2 Average loss: 159.5319
====> Test set loss: 155.8455
Train Epoch: 3 [0/60000 (0%)]	Loss: 155.014248
Train Epoch: 3 [10000/60000 (17%)]	Loss: 146.828193
Train Epoch: 3 [20000/60000 (33%)]	Loss: 149.715059
Train Epoch: 3 [30000/60000 (50%)]	Loss: 154.014395
Train Epoch: 3 [40000/60000 (67%)]	Loss: 155.38

In [137]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()  # 64 组 mu + log_var
    sample = vae.decoder(z).cuda()  # output should be (64,784)
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')